# Variance-level plots

### Classes and modules

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# plt.rcParams["lines.color"] = "w"
# plt.rcParams["text.color"] = "w"
# plt.rcParams["axes.labelcolor"] = "w"
# plt.rcParams["xtick.color"] = "w"
# plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

import pycuda.driver as cuda

GPU Ocean-modules:

In [2]:
from gpuocean.utils import IPythonMagic, Common

In [3]:
%cuda_context_handler gpu_ctx

In [4]:
ls = [6, 7, 8, 9]

In [5]:
from gpuocean.utils import DoubleJetCase

args_list = []
init_list = []

for l in ls:
    doubleJetCase = DoubleJetCase.DoubleJetCase(gpu_ctx, DoubleJetCase.DoubleJetPerturbationType.SteadyState, ny=2**l, nx=2**(l+1))
    doubleJetCase_args, doubleJetCase_init, _ = doubleJetCase.getInitConditions()

    args_list.append(doubleJetCase_args)
    init_list.append(doubleJetCase_init)

### Read variances from file

In [6]:
# source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-09-26T11_10_32"
# source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-09-25T13_55_37"
source_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/VarianceLevelsDA/2023-10-27T17_43_20"

In [7]:
vars = np.load(source_path+"/vars_"+str(10*24*3600)+".npy")
diff_vars = np.load(source_path+"/diff_vars_"+str(10*24*3600)+".npy")

In [8]:
vars, diff_vars

(array([[1.5858183e+02, 1.5990373e+11, 9.6634626e+10],
        [9.7177979e+02, 3.1332507e+12, 8.6894812e+11],
        [2.5992576e+03, 5.8559064e+12, 3.7920312e+12],
        [4.5634897e+03, 8.8159583e+12, 6.7380870e+12]], dtype=float32),
 array([[7.2866943e+02, 2.6328946e+12, 6.9889930e+11],
        [1.7286455e+03, 3.1621387e+12, 2.2114027e+12],
        [2.4873433e+03, 2.0927904e+12, 1.6598405e+12]], dtype=float32))

### Read work from file

In [9]:
work_path = "/home/florianb/havvarsel/multilevelDA/doublejet/scripts/PracticalCost/2023-09-18T12_54_47"

In [10]:
works = np.mean(np.load(work_path+"/costsPure.npy"), axis=-1)
diff_works = np.mean(np.load(work_path+"/costsPartnered.npy"), axis=-1)

Use theoretical work

In [11]:
works = [1/(8**3), 1/(8**2), 1/(8), 1]
diff_works = [0, 1/(8**2), 1/(8), 1]

In [12]:
works, diff_works

([0.001953125, 0.015625, 0.125, 1], [0, 0.015625, 0.125, 1])

## Ensembles with equal work

In [13]:
SL_Ne = 50

In [14]:
SL_Ne2 = works[-1]*SL_Ne/works[-2]
SL_Ne2

400.0

In [15]:
SL_Ne3 = works[-1]*SL_Ne/works[-3]
SL_Ne3

3200.0

In [16]:
SL_Ne4 = works[-1]*SL_Ne/works[-4]
SL_Ne4

25600.0

In [17]:
from gpuocean.utils import MultiLevelAnalysis

In [18]:
analysis = MultiLevelAnalysis.Analysis(args_list, vars, diff_vars, works, diff_works)

In [19]:
# ML_Ne = analysis.optimal_Ne(1.425e-1)
ML_Ne = analysis.optimal_Ne(1.05e-1)
ML_Ne

array([264, 270, 132,  46], dtype=int32)

In [20]:
analysis.work(ML_Ne)/analysis.works[-1]

67.234375

In [21]:
analysis1 = MultiLevelAnalysis.Analysis(args_list[1:], vars[1:], diff_vars[1:], works[1:], diff_works[1:])

In [22]:
# ML_Ne1 = analysis1.optimal_Ne(1.3e-1)
ML_Ne1 = analysis1.optimal_Ne(1.05e-1)
ML_Ne1

array([300, 131,  46], dtype=int32)

In [23]:
analysis1.work(ML_Ne1)/analysis.works[-1]

67.0625

In [24]:
analysis2 = MultiLevelAnalysis.Analysis(args_list[2:], vars[2:], diff_vars[2:], works[2:], diff_works[2:])

In [25]:
# ML_Ne2 = analysis2.optimal_Ne(1.225e-1)
ML_Ne2 = analysis2.optimal_Ne(1.12e-1)
ML_Ne2

array([150,  41], dtype=int32)

In [26]:
analysis2.work(ML_Ne2)/analysis.works[-1]

59.75

### Comparisons

In [27]:
# works = [1/(8**2), 1/(8), 1]
# diff_works = [1/(8), 1]

SpeedUp

In [28]:
# var_idx, tau = 0, 5e-2
var_idx, tau = 1, 5e3
# var_idx, tau = 2, 5e3

allwork = 0
for l_idx in range(len(ls)):
    if l_idx == 0: 
        allwork += np.sqrt(vars[l_idx][var_idx] * works[l_idx])
    else:
        allwork += np.sqrt(diff_vars[l_idx-1][var_idx] * diff_works[l_idx])

optNe_ref = np.zeros(len(ls))
for l_idx in range(len(ls)):
    if l_idx == 0: 
        optNe_ref[l_idx] = np.sqrt(vars[l_idx][var_idx]/works[l_idx]) * allwork
    else: 
        optNe_ref[l_idx] = np.sqrt(diff_vars[l_idx-1][var_idx]/diff_works[l_idx]) * allwork

Ne = np.int32(np.ceil(1/(tau**2)*optNe_ref))
print("Ne = ", Ne)

MLens_work = Ne[0]*works[0]
for l_idx in range(1, len(ls)):
    MLens_work += Ne[l_idx]*diff_works[l_idx]

SLens_work = vars[-1][var_idx]/(tau**2) * works[-1]

print("SpeedUp = ", SLens_work/MLens_work)

Ne =  [ 830937 1192094  461891  132852]
SpeedUp =  1.6725577007945287


Theoretical error

In [50]:
np.sqrt(vars[-1])/np.sqrt(25)

array([1.3510721e+01, 5.9383362e+05, 5.1915650e+05], dtype=float32)

In [29]:
np.sqrt(vars[-1])/np.sqrt(SL_Ne)

array([9.5535221e+00, 4.1990375e+05, 3.6709909e+05], dtype=float32)

In [44]:
np.sqrt(vars[-1])/np.sqrt(100)

array([6.7553606e+00, 2.9691681e+05, 2.5957825e+05], dtype=float32)

In [42]:
analysis.theoretical_error([622,262,99,32])

array([9.9109373e+00, 3.2809572e+05, 2.7754341e+05], dtype=float32)

In [48]:
analysis.work([622,262,99,32])

49.68359375

In [49]:
analysis.works

[0.001953125, 0.015625, 0.125, 1]

In [30]:
analysis.theoretical_error(ML_Ne)

array([8.3945189e+00, 2.8250344e+05, 2.3620139e+05], dtype=float32)

In [31]:
analysis1.theoretical_error(ML_Ne1)

array([8.3968868e+00, 2.8298069e+05, 2.3634916e+05], dtype=float32)

In [32]:
analysis2.theoretical_error(ML_Ne2)

array([8.8314943e+00, 3.0013838e+05, 2.5644517e+05], dtype=float32)

In [33]:
vars

array([[1.5858183e+02, 1.5990373e+11, 9.6634626e+10],
       [9.7177979e+02, 3.1332507e+12, 8.6894812e+11],
       [2.5992576e+03, 5.8559064e+12, 3.7920312e+12],
       [4.5634897e+03, 8.8159583e+12, 6.7380870e+12]], dtype=float32)

In [34]:
ML_Ne

array([264, 270, 132,  46], dtype=int32)

In [35]:
np.sqrt(vars[-1][2]/50)

367099.08705220174

In [36]:
np.sqrt(vars[0][2]/ML_Ne[0])

19132.178396142404

In [37]:
np.sqrt(diff_vars[0][2]/ML_Ne[1])

50877.45991330178

In [38]:
np.sqrt(diff_vars[1][2]/ML_Ne[2])

129433.57790563678

In [39]:
np.sqrt(diff_vars[2][2]/ML_Ne[3])

189956.5451231687

In [40]:
ML_Ne2

array([150,  41], dtype=int32)